In [41]:
import psycopg2
import pyarrow as pa
from pyarrow import parquet
import pandas as pd
import boto3

In [45]:
# PostgreSQL connection parameters
postgres_params = {
    "host": "localhost",
    "port": 5432,
    "database": "ny_taxi",
    "user": "root",
    "password": "root",
}
try:
    # Attempt to connect to PostgreSQL
    conn = psycopg2.connect(**postgres_params)
    print("Connection to PostgreSQL successful!")
    # Close the connection
    conn.close()
except psycopg2.Error as e:
    print("Error connecting to PostgreSQL:", e)

Connection to PostgreSQL successful!


In [50]:
# Create a session with the specified profile
session = boto3.Session(profile_name='bdc-aws-account')

# Create an S3 client using session info
s3_client = session.client('s3')

try:
    # List objects in the bucket
    response = s3_client.list_objects_v2(Bucket="bdc-de-build-your-own-stack")

    # Check if listing was successful
    if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
        print("Connection to S3 successful!")
    else:
        print("Error connecting to S3:", response)
except Exception as e:
    print("Error connecting to S3:", e)

Connection to S3 successful!


In [51]:
# S3 connection parameters
s3_bucket_name = "bdc-de-build-your-own-stack"
s3_prefix = "batch-data-ingestion/sean-meehan/"

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**postgres_params)

    # Query to extract first 1000 rows from yellow_taxi_data table
    query = "SELECT * FROM public.yellow_taxi_data LIMIT 1000;"

    # Execute the query and fetch the results into a DataFrame
    df = pd.read_sql(query, conn)

    # Close the PostgreSQL connection
    conn.close()

    # Write the DataFrame to a Parquet file
    parquet_file = "/tmp/yellow_taxi_data_first_1000_rows.parquet"
    df.to_parquet(parquet_file)

    # Upload the Parquet file to S3
    with open(parquet_file, 'rb') as data:
        s3_client.upload_fileobj(data, s3_bucket_name, s3_prefix + "yellow_taxi_data_first_1000_rows.parquet")
        
    print("Parquet file uploaded to S3 successfully!")

except Exception as e:
    print("Error:", e)

/var/folders/hx/wwq6j7ln1fzb__7sk19yd3kr0000gq/T/ipykernel_41239/2478931160.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Parquet file uploaded to S3 successfully!
